Paul Giesting, Project 2

### The final notebook?

Bring in hitting/speed/small ball data, pitching data, fielding data, attendance data.

##### Multiple strategies:

##### Atemporal regression.
Do a standard randomized split on the data.
Train up a polynomial, degree 2 LASSO model on selected parameters from visualizations.
Test model to see if it holds water.
Look at coefficients & suggest interventions to reverse attendance declines.

##### Time series.
Split data into 20th & 21st century, train & test.
Train up a similar model.
See how much adding the time factor adds to the fit (representing external cultural changes).
Change train / test splits as needed.
Project the 2020s with various rules / culture interventions.

In [41]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [5]:
with open('bat_rate_data.pkl','rb') as cellar:
    bat_df = pickle.load(cellar)
with open('pitch_rate_data.pkl','rb') as cellar:
    pitch_df = pickle.load(cellar)
with open('field_rate_data.pkl','rb') as cellar:
    field_df = pickle.load(cellar)
# with open('sp.pkl','rb') as cellar:
#    sp_df = pickle.load(cellar)
# with open('sb.pkl','rb') as cellar:
#    sb_df = pickle.load(cellar)
with open('att.pkl','rb') as cellar:
    att_df = pickle.load(cellar)

###### Target:
* NormAtt/G * 1000000 (or Att)
= attendance per game in units of parts per million of the U.S. population (using ppm because as a fraction it's really small and seaborn actually gagged on it in some of the pairplots & treated it as zero); the calculation was already done in the visualization notebooks

###### Flagship parameters for model:
* BA, SLG, HR/G, 'SB/G', SH/G, SO9, ERA, DefEff, GmDur

###### "On-deck" parameters:
* IBB/G, 'CS/G', '3B/G', SF/G, DP/G, A/G, Fld% 

In [7]:
bat_df.columns

Index(['Year', 'G', 'R/G', 'PA/G', 'AB/G', 'H/G', '2B/G', '3B/G', 'HR/G',
       'RBI/G', 'SB/G', 'CS/G', 'BB/G', 'SO/G', 'BA', 'OBP', 'SLG', 'OPS',
       'OPS+', 'TB/G', 'GDP/G', 'HBP/G', 'SH/G', 'SF/G', 'IBB/G', 'LOB/G'],
      dtype='object')

In [8]:
pitch_df.columns

Index(['Year', 'ERA', 'HR9', 'BB9', 'SO9', 'SO/W', 'WHIP', 'Att'], dtype='object')

In [9]:
field_df.columns

Index(['Year', 'RA/G', 'DefEff', 'Fld%', 'DP/G', 'PO/G', 'A/G', 'Att'], dtype='object')

In [10]:
att_df.columns

Index(['Year', 'ASG Ratings', 'WS Ratings', 'Pitches/PA', 'Pitchers/G',
       'GmDur', 'NormAtt/G'],
      dtype='object')

In [30]:
# data to feed to train_test
y = pd.DataFrame(pitch_df['Att'])
X1 = [bat_df['BA'],bat_df['SLG'],bat_df['HR/G'],
     bat_df['SB/G'],bat_df['SH/G'],pitch_df['SO9'],
     pitch_df['ERA'],field_df['DefEff'],att_df['GmDur']]
X1 = pd.DataFrame(X1)
X1 = X1.transpose().fillna(0.0)
X2 = [bat_df['IBB/G'],bat_df['CS/G'],bat_df['3B/G'],
     bat_df['SF/G'],field_df['DP/G'],field_df['A/G'],
     field_df['Fld%']]
X2 = pd.DataFrame(X2)
X2 = X2.transpose().fillna(0.0)

In [31]:
print(len(y), len(X1))

119 119


In [32]:
X1.dtypes

BA        float64
SLG       float64
HR/G      float64
SB/G      float64
SH/G      float64
SO9       float64
ERA       float64
DefEff    float64
GmDur     float64
dtype: object

In [33]:
y.dtypes

Att    float64
dtype: object

In [38]:
# a straight linear regression as a baseline
for i in range(5):
    X1tr, X1v, y1tr, y1v = train_test_split(X1, y, test_size=0.3)
    lin_atemp = LinearRegression()
    lin_atemp.fit(X1tr,y1tr)
    print(lin_atemp.score(X1tr,y1tr),lin_atemp.score(X1v,y1v))

0.6792762088484843 0.7058645840909972
0.6552029963641688 0.7378659809883454
0.7069743879238279 0.5829031617386146
0.7665291046169564 0.47463951668425985
0.6721885251481028 0.6149453514893122


This model is unstable; I guess I'm not surprised, with only 119 data points. Hopefully I don't have to go back and rethink things completely and try to pull data at the team... or player... level. I don't know that I would have any attendance data to work with.

Part of the problem is likely that I'm throwing messy, random variables into the regression that probably really are unrelated to attendance trends. I can do three things about this:
* Lasso may be able to prune the crap variables in an automated way.
* I'll do Ridge since it's there.
* If I have to, I can go back to my visualizations and prune variables manually.

Beyond that, there is polynomial feature selection. Forgot about that. There would be a blizzard of features at that point, something like as many features as lines of data, if I stick all of X1 in there, so I'll need to prune that down regardless.

In [43]:
scaler = StandardScaler()

In [45]:
# Lasso
for i in range(5):
    X1tr, X1v, y1tr, y1v = train_test_split(X1, y, test_size=0.3)
    X1sc = scaler.fit_transform(X1tr.values)
    X1vsc = scaler.fit_transform(X1v.values)
    lasso_atemp = LassoCV()
    lasso_atemp.fit(X1tr,y1tr)
    print(lasso_atemp.score(X1tr,y1tr),lasso_atemp.score(X1v,y1v))

/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6019383806041514 0.6639735286701927
0.672115618775148 0.584476793969793
0.6078180994904796 0.688370159856518
0.6279818970298718 0.6793406597396442
0.6606955308072073 0.5569604612287644


/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Ugh. That's more stable... and quite bad. I got tons better results for using the StandardScaler, though. Without it, there was a 0.2 hit between train & validate.

In [46]:
# Ridge
for i in range(5):
    X1tr, X1v, y1tr, y1v = train_test_split(X1, y, test_size=0.3)
    X1sc = scaler.fit_transform(X1tr.values)
    X1vsc = scaler.fit_transform(X1v.values)
    ridge_atemp = RidgeCV()
    ridge_atemp.fit(X1tr,y1tr)
    print(ridge_atemp.score(X1tr,y1tr),ridge_atemp.score(X1v,y1v))

0.6792963922104938 0.4498004858259206
0.5605820644447778 0.718955191306363
0.6925559612920577 0.5032934949253027
0.7066609591973361 0.4465464549284093
0.7103109395200264 0.4935382505629421


That's just terrible.

Let me run the Lasso code again and look at the coefficients.

In [53]:
# Lasso
for i in range(5):
    X1tr, X1v, y1tr, y1v = train_test_split(X1, y, test_size=0.3)
    X1sc = scaler.fit_transform(X1tr.values)
    X1vsc = scaler.fit_transform(X1v.values)
    lasso_atemp = LassoCV()
    lasso_atemp.fit(X1tr,y1tr)
    out = pd.DataFrame(lasso_atemp.coef_)
    out = out.transpose().round(decimals=2)
    out.columns=X1tr.columns
    print(out)
    print(lasso_atemp.score(X1tr,y1tr),lasso_atemp.score(X1v,y1v),'\n')

/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    BA  SLG   HR/G   SB/G  SH/G   SO9   ERA  DefEff  GmDur
0  0.0  0.0  28.69  13.13  -0.0  2.39  5.09     0.0   4.71
0.689999590261992 0.4864431137594343 

    BA  SLG   HR/G   SB/G  SH/G   SO9   ERA  DefEff  GmDur
0 -0.0 -0.0  53.23  24.09  9.04 -1.56  1.35     0.0   7.72
0.6662049265330816 0.5607370846601449 

    BA  SLG   HR/G  SB/G  SH/G   SO9   ERA  DefEff  GmDur
0 -0.0 -0.0  53.25  20.2  -4.2 -2.58 -1.06     0.0   5.88
0.6467489025532706 0.6329747735390672 



/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    BA  SLG  HR/G  SB/G  SH/G   SO9   ERA  DefEff  GmDur
0  0.0  0.0   0.0   0.0  -0.0  4.04  9.16     0.0   8.12
0.6649292700812106 0.45723291527650134 

    BA  SLG   HR/G   SB/G  SH/G   SO9   ERA  DefEff  GmDur
0  0.0  0.0  45.15  15.02  0.12 -1.81  2.88     0.0   6.71
0.6561123685589678 0.612075993994247 



Features with consistent coefficients:
* HR/G - when it shows up, always positive. (Chicks dig the long ball.)
* SB/G - when it shows up, always positive.
* GmDur - humorously, always positive. (So much for all the whining about games being too long, right?)

In [55]:
matrix = pd.concat([y,X1,X2],axis=1)
matrix.corr()

,Att,BA,SLG,HR/G,SB/G,SH/G,SO9,ERA,DefEff,GmDur,IBB/G,CS/G,3B/G,SF/G,DP/G,A/G,Fld%
Att,1.000000,-0.074802,0.599337,0.768460,-0.255794,-0.677280,0.680903,0.538646,0.148423,0.726195,0.673738,0.305263,-0.744548,0.710865,0.427248,-0.750937,0.762879
BA,-0.074802,1.000000,0.535423,-0.047638,-0.185206,0.259055,-0.406064,0.612650,-0.643326,-0.246444,-0.428885,0.191441,0.422027,-0.304458,0.335200,0.182241,-0.110145
SLG,0.599337,0.535423,1.000000,0.811275,-0.526245,-0.572893,0.486509,0.972783,-0.284641,0.539419,0.284321,0.274243,-0.459878,0.475003,0.661502,-0.696457,0.683670
HR/G,0.768460,-0.047638,0.811275,1.000000,-0.530631,-0.878449,0.828725,0.735752,0.155838,0.816311,0.685382,0.220729,-0.859243,0.798303,0.606309,-0.952053,0.900300
SB/G,-0.255794,-0.185206,-0.526245,-0.530631,1.000000,0.475685,-0.123513,-0.584424,-0.401711,-0.339720,-0.412567,-0.116524,0.394269,-0.165932,-0.845177,0.564735,-0.619917
SH/G,-0.677280,0.259055,-0.572893,-0.878449,0.475685,1.000000,-0.784779,-0.538054,-0.290192,-0.788490,-0.778570,-0.155089,0.906341,-0.781008,-0.534124,0.930960,-0.824229
SO9,0.680903,-0.406064,0.486509,0.828725,-0.123513,-0.784779,1.000000,0.350561,0.086218,0.758921,0.594626,0.106170,-0.850030,0.810498,0.138733,-0.847510,0.741285
ERA,0.538646,0.612650,0.972783,0.735752,-0.584424,-0.538054,0.350561,1.000000,-0.278393,0.475076,0.245514,0.250899,-0.392443,0.354990,0.726598,-0.642773,0.628767
DefEff,0.148423,-0.643326,-0.284641,0.155838,-0.401711,-0.290192,0.086218,-0.278393,1.000000,0.299439,0.624083,0.089356,-0.440751,0.279398,0.317716,-0.247642,0.375841
GmDur,0.726195,-0.246444,0.539419,0.816311,-0.339720,-0.788490,0.758921,0.475076,0.299439,1.000000,0.735879,0.366122,-0.812189,0.773901,0.497488,-0.826863,0.796635


Features with |corr| > 0.74 with Att:
* HR/G
* A/G
* Fld%
* 3B/G

Features with 0.67 < |corr| < 0.74 with Att:
* GmDur
* SF/G
* SH/G
* SO9
* IBB/G

I really want to stick a pitching stat into the mix, so I will bump A/G (what the heck do assists have to do with anything anyway?) and put SO9 in. That's the four features I want for polynomial regression.

I realize that precisely because there is probably a polynomial relationship between a number of these variables, the linear correlation coefficients may be bad for important variables, but this is where I'll start. I wish I could just throw six or ten features in, poly=2 them, and regress against 10,000 data points, but I've backed myself into this problem, still suspect it's worth trying, and I will do what I can. After this first cut I will consult the pairplots in my visualization notebooks again to look for arched data.

In [56]:
Xp = pd.concat([X1['HR/G'],X1['SO9'],X2['Fld%'],X2['3B/G']],axis=1)
Xp.head(2)

,HR/G,SO9,Fld%,3B/G
1901,0.201439,3.2,0.943,0.553957
1902,0.157143,3.0,0.949,0.435714


In [57]:
pf = PolynomialFeatures(degree=2)
pf.fit(Xp)
pf.get_feature_names(input_features=list(Xp.columns))

['1',
 'HR/G',
 'SO9',
 'Fld%',
 '3B/G',
 'HR/G^2',
 'HR/G SO9',
 'HR/G Fld%',
 'HR/G 3B/G',
 'SO9^2',
 'SO9 Fld%',
 'SO9 3B/G',
 'Fld%^2',
 'Fld% 3B/G',
 '3B/G^2']

In [58]:
Xparr = pf.transform(Xp)
Xparr

array([[1.        , 0.20143885, 3.2       , ..., 0.889249  , 0.52238129,
        0.30686817],
       [1.        , 0.15714286, 3.        , ..., 0.900601  , 0.41349286,
        0.18984694],
       [1.        , 0.15107914, 3.6       , ..., 0.900601  , 0.49839568,
        0.27581388],
       ...,
       [1.        , 1.25925926, 8.3       , ..., 0.968256  , 0.164     ,
        0.02777778],
       [1.        , 1.14814815, 8.5       , ..., 0.968256  , 0.17007407,
        0.02987349],
       [1.        , 1.39506173, 8.9       , ..., 0.968256  , 0.15792593,
        0.02575827]])

In [65]:
Xparr_cols = pf.get_feature_names(input_features=list(Xp.columns))
Xpp = pd.DataFrame(Xparr,columns=Xparr_cols,index=Xp.index)
Xpp.head(2)

,1,HR/G,SO9,Fld%,3B/G,HR/G^2,HR/G SO9,HR/G Fld%,HR/G 3B/G,SO9^2,SO9 Fld%,SO9 3B/G,Fld%^2,Fld% 3B/G,3B/G^2
1901,1.0,0.201439,3.2,0.943,0.553957,0.040578,0.644604,0.189957,0.111588,10.24,3.0176,1.772662,0.889249,0.522381,0.306868
1902,1.0,0.157143,3.0,0.949,0.435714,0.024694,0.471429,0.149129,0.068469,9.00,2.8470,1.307143,0.900601,0.413493,0.189847


In [60]:
for i in range(5):
    Xptr, Xpv, yptr, ypv = train_test_split(Xpp, y, test_size=0.25)
    Xpsc = scaler.fit_transform(Xptr.values)
    Xpvsc = scaler.fit_transform(Xpv.values)
    lasso_atemp = LassoCV()
    lasso_atemp.fit(Xpsc,yptr)
    out = pd.DataFrame(lasso_atemp.coef_)
    out = out.transpose().round(decimals=2)
    out.columns=Xpp.columns
    print(out)
    print(lasso_atemp.score(Xpsc,yptr),lasso_atemp.score(Xpvsc,ypv),'\n')

/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.337658921011098, tolerance: 3.691606365294727
  tol, rng, random, positive)
/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.020675798477896, tolerance: 3.691606365294727
  tol, rng, random, positive)
/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Object

/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.4705756450857734, tolerance: 3.218016417428933
  tol, rng, random, positive)
/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.4634580080710293, tolerance: 3.218016417428933
  tol, rng, random, positive)
/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.4222651809577656, tolerance: 3.218016417428933
  tol, rng, random, positive)
/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: O

     1   HR/G   SO9  Fld%  3B/G  HR/G^2  HR/G SO9  HR/G Fld%  HR/G 3B/G  \
0  0.0  16.67  9.77  0.88   1.1   11.96    -65.06      55.11     -23.38   

   SO9^2  SO9 Fld%  SO9 3B/G  Fld%^2  Fld% 3B/G  3B/G^2  
0    4.2      7.48     -5.24   10.27       3.69   17.21  
0.7121953373900252 0.543097903772974 



/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.771193555567152, tolerance: 3.6077438878707895
  tol, rng, random, positive)
/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.6520638317069825, tolerance: 3.6077438878707895
  tol, rng, random, positive)
/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.881528977697599, tolerance: 3.6077438878707895
  tol, rng, random, positive)
/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: 

     1  HR/G   SO9  Fld%  3B/G  HR/G^2  HR/G SO9  HR/G Fld%  HR/G 3B/G  SO9^2  \
0  0.0  6.55  10.6  0.62  -0.0   13.02    -50.88      40.21     -14.35  -0.49   

   SO9 Fld%  SO9 3B/G  Fld%^2  Fld% 3B/G  3B/G^2  
0      8.94      -2.6   10.13       -0.0   13.15  
0.6694582200351662 0.728054558775822 



...

     1   HR/G   SO9  Fld%  3B/G  HR/G^2  HR/G SO9  HR/G Fld%  HR/G 3B/G  \
0  0.0  19.78  8.92  1.05  -0.0     0.0    -26.42      25.87     -12.31   

   SO9^2  SO9 Fld%  SO9 3B/G  Fld%^2  Fld% 3B/G  3B/G^2  
0 -21.78     27.16     -4.38    7.03       -0.0   19.19  
0.7292128096198075 0.4266963840385261 



/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 93.65511745427739, tolerance: 3.826237083440044
  tol, rng, random, positive)
/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 95.58322671564474, tolerance: 3.826237083440044
  tol, rng, random, positive)
/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 95.84429183034445, tolerance: 3.826237083440044
  tol, rng, random, positive)
/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Obje

     1   HR/G   SO9  Fld%  3B/G  HR/G^2  HR/G SO9  HR/G Fld%  HR/G 3B/G  \
0  0.0  24.19  6.15   0.0  -0.0     0.0    -38.55      45.19     -21.52   

   SO9^2  SO9 Fld%  SO9 3B/G  Fld%^2  Fld% 3B/G  3B/G^2  
0   -0.0      2.23     -2.31    10.2       -0.0    21.2  
0.7049374862568927 0.15945528007001708 



/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11.540679606976482, tolerance: 3.7072655092887286
  tol, rng, random, positive)
/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11.035128880725097, tolerance: 3.7072655092887286
  tol, rng, random, positive)
/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.543322701967554, tolerance: 3.7072655092887286
  tol, rng, random, positive)
/home/paulus/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning

     1  HR/G   SO9  Fld%  3B/G  HR/G^2  HR/G SO9  HR/G Fld%  HR/G 3B/G  SO9^2  \
0  0.0  9.82  11.2  0.24  -0.0   10.88    -53.59      44.01     -15.27   -0.0   

   SO9 Fld%  SO9 3B/G  Fld%^2  Fld% 3B/G  3B/G^2  
0      9.05     -3.27    8.04       -0.0   14.24  
0.6800180634632638 0.6879827186954844 



It converged once out of five times. Delightful. At least for that model, the train and test splits gave effectively identical scores. I *might* be getting somewhere.

I want p values for each polynomial feature to see what that looks like. The coefficients themselves only tell me a little. I can look at some stuff from statsmodels for an interactive model.

In [67]:
import statsmodels.api as sm
logit_model=sm.Logit(y,Xpp)
result=logit_model.fit()
print(result.summary())

ValueError: endog must be in the unit interval.

Or effing not.